# RAG Model using Langchain library

### Installation of libraries

In [ ]:
%pip install -q qdrant-client langchain

### Read openai key

In [ ]:
# load openai api key
import os

from dotenv import load_dotenv
load_dotenv()

if not 'OPENAI_API_KEY' in os.environ:
    raise ValueError('OPENAI_API_KEY is not set')

## Dataset Loader

In [ ]:
# Import the sample dataset from the library
from validmind.datasets.llm.rag import rfp

raw_df = rfp.load_data()
train_df, test_df = rfp.preprocess(raw_df)

In [ ]:
import validmind as vm

vm_train_ds = vm.init_dataset(
    train_df,
    text_column="question",
    target_column = "ground_truth",
    __log=False
)

vm_test_ds = vm.init_dataset(
    test_df,
    text_column="question",
    target_column = "ground_truth",
    __log=False
)

test_df.head()

## Embedding Model Selection

First let's setup our embedding model and run some tests to make sure its working well.

In [ ]:
from langchain_openai import OpenAIEmbeddings

from validmind.models import FunctionModel

embedding_client = OpenAIEmbeddings(model="text-embedding-3-small")

def embed(input):
    """Returns a text embedding for the given text"""
    return embedding_client.embed_query(input["question"])

vm_embedder = FunctionModel(input_id="embedding_model", predict_fn=embed)

In [ ]:
vm_test_ds.assign_predictions(vm_embedder)
print(vm_test_ds)

### Run tests

In [ ]:
from validmind.tests import run_test

result = run_test(
    "validmind.model_validation.embeddings.StabilityAnalysisRandomNoise",
    inputs={"model": vm_embedder, "dataset": vm_test_ds},
    params={"probability": 0.3},
)

## Setup Vector Store

#### Generate embeddings for the questions

In [ ]:
vm_train_ds.assign_predictions(vm_embedder)
print(vm_train_ds)

#### Insert embeddings and questions into Vector DB

In [ ]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DataFrameLoader

# load documents from dataframe
loader = DataFrameLoader(train_df, page_content_column="question")
docs = loader.load()
# choose model using embedding client
embedding_client = OpenAIEmbeddings(model="text-embedding-3-small")

# setup vector datastore
qdrant = Qdrant.from_documents(
    docs,
    embedding_client,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="rfp_rag_collection",
)

## Setup Retrieval Model

In [ ]:
def retrieve(input):
    contexts = []

    for result in qdrant.similarity_search_with_score(input["question"]):
        document, score = result
        context = f"Q: {document.page_content}\n"
        context += f"A: {document.metadata['ground_truth']}\n"

        contexts.append(context)

    return contexts


vm_retriever = FunctionModel(input_id="retrieval_model", predict_fn=retrieve)

In [ ]:
vm_test_ds.assign_predictions(model=vm_retriever)
print(vm_test_ds)

## Setup Generation Model

In [ ]:
from openai import OpenAI


system_prompt = """
You are an expert RFP AI assistant.
You are tasked with answering new RFP questions based on existing RFP questions and answers.
You will be provided with the existing RFP questions and answer pairs that are the most relevant to the new RFP question.
After that you will be provided with a new RFP question.
You will generate an answer and respond only with the answer.
Ignore your pre-existing knowledge and answer the question based on the provided context.
""".strip()

openai_client = OpenAI()

def generate(input):
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "\n\n".join(input["retrieval_model"])},
            {"role": "user", "content": input["question"]},
        ],
    )

    return response.choices[0].message.content

vm_generator = FunctionModel(input_id="generation_model", predict_fn=generate)

In [ ]:
import pandas as pd

vm_generator.predict(pd.DataFrame({"retrieval_model": [["My name is anil"]], "question": ["what is my name"]}))

## Setup RAG Model (Pipeline of "Component" Models)

In [ ]:
from validmind.models import PipelineModel

vm_rag_model = PipelineModel(vm_retriever | vm_generator, input_id="rag_model")

In [ ]:
vm_test_ds.assign_predictions(model=vm_rag_model)
print(vm_test_ds)

# Run tests

In [ ]:
import plotly.express as px

def plot_distribution(scores):
    # plot distribution of scores (0-1) from ragas metric
    # scores is a list of floats
    fig = px.histogram(x=scores, nbins=10)
    fig.show()

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
result = vm.tests.run_test(
    "validmind.model_validation.ragas.AnswerSimilarity",
    inputs={"dataset": vm_test_ds},
    show=False,
)
plot_distribution(result.metric.summary.results[0].data)

In [ ]:
result = run_test(
    "validmind.model_validation.ragas.ContextEntityRecall",
    inputs={"dataset": vm_test_ds},
    show=False,
)
plot_distribution(result.metric.summary.results[0].data)

In [ ]:
result = run_test(
    "validmind.model_validation.ragas.ContextPrecision",
    inputs={"dataset": vm_test_ds},
    show=False,
)
 
plot_distribution(result.metric.summary.results[0].data)

In [ ]:
result = run_test(
    "validmind.model_validation.ragas.ContextRelevancy",
    inputs={"dataset": vm_test_ds},
    show=False,
)
plot_distribution(result.metric.summary.results[0].data)